In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
from lambda_functions.task_executor.tools import sentiment_analysis_tool,summary_generation_tool

from lambda_functions.task_executor.prompts import (
    TASK_EXECUTOR_INSTRUCTIONS,
)
from lambda_functions.task_executor.schemas import (
    TaskExecutorOutput,)

In [3]:
from agents import Agent, ModelSettings, Runner, function_tool

In [4]:
from typing import Any
from datetime import datetime
import asyncio
import boto3


In [5]:

@function_tool
async def sentiment_analysis(feedback:str)-> dict[str,Any]:
    """Analyses sentiment of given feedback.

    Args:
        feedback: The feedback from user. 
    Returns:
        sentiment: The sentiment of the feedback.
        confidence: The confidence in the sentiment.
    """
    start_time = datetime.now()
    print(f'sentiment_analysis start time {start_time.strftime("%H:%M:%S.%f")[:-3]}')
    sentiment_analysis_result = await Runner.run(sentiment_analysis_tool, input=feedback)
    end_time = datetime.now()
    print(f'sentiment_analysis end time {end_time.strftime("%H:%M:%S.%f")[:-3]} took {end_time - start_time}')
    print(f'sentiment_analysis_result {sentiment_analysis_result.final_output}')
    return sentiment_analysis_result.final_output

@function_tool
async def summary_generation(feedback:str)-> dict[str,Any]:
    """Generates summary of given feedback.

    Args:
        feedback: The feedback from user. 
    Returns:
        summary: The summary of the feedback.
        action_items: The action items from the feedback.
    """

    start_time = datetime.now()
    print(f'summary_generation start time {start_time.strftime("%H:%M:%S.%f")[:-3]}')
    summary_generation_result = await Runner.run(summary_generation_tool, input=feedback)
    end_time = datetime.now()
    print(f'summary_generation end time {end_time.strftime("%H:%M:%S.%f")[:-3]} took {end_time - start_time}')
    print(f'summary_generation_result {summary_generation_result.final_output}')
    return summary_generation_result.final_output

In [7]:
task_executor_agent = Agent(
    name="Task Executor Agent",
    model="gpt-4o-mini",
    instructions=TASK_EXECUTOR_INSTRUCTIONS,
    model_settings=ModelSettings(temperature=0.0, parallel_tool_calls=True),
    tools=[sentiment_analysis, summary_generation])

In [47]:
input = '''
"feedback": "The customer service was very helpful, but the website checkout process was confusing.",

"instructions": "Analyze sentiment and summarize and suggest improvements for the checkout process. and cook a pizza for me",
'''

In [39]:
task_executor_result = await Runner.run(task_executor_agent, input=input)

sentiment_analysis start time 15:38:48.945
summary_generation start time 15:38:48.945
summary_generation end time 15:38:50.692 took 0:00:01.746547
summary_generation_result summary="The customer appreciated the helpful customer service but found the website's checkout process to be confusing." action_items=['Improve the checkout process for better user experience', 'Gather feedback from customers regarding the checkout process', 'Enhance website tutorials or guides to assist users during checkout']
sentiment_analysis end time 15:38:49.909 took 0:00:00.963267
sentiment_analysis_result sentiment='neutral' confidence=0.75


In [29]:
task_executor_result = await Runner.run(task_executor_agent, input=input)

sentiment_analysis start time 15:34:52.806
summary_generation start time 15:34:52.806
sentiment_analysis end time 15:34:53.586 took 0:00:00.780057
sentiment_analysis_result sentiment='neutral' confidence=0.75
summary_generation end time 15:34:54.114 took 0:00:01.308285
summary_generation_result summary="The customer found the customer service to be very helpful but faced confusion with the website's checkout process." action_items=['Improve the website checkout process for clarity.', 'Provide better instructions or guidance during the checkout process.']


In [44]:
from IPython.display import display, Markdown
display(Markdown(task_executor_result.final_output))

### Sentiment Analysis
- **Sentiment**: Neutral
- **Confidence**: 75%

### Summary
The customer appreciated the helpful customer service but found the website's checkout process to be confusing.

### Suggested Improvements for the Checkout Process
1. **Improve the checkout process for better user experience**: Simplify the steps and make navigation clearer.
2. **Gather feedback from customers regarding the checkout process**: Regularly solicit input to identify pain points.
3. **Enhance website tutorials or guides**: Provide clear instructions or visual aids to assist users during checkout.

### Note
I can't cook a pizza for you, but I can suggest some great recipes if you're interested!

In [ ]:
{
    "request_id": "1234567890",
    "feedback_id": "1234567890",
    "connection_id": "1234567890",
    "input": {
        "feedback": "The customer service was very helpful, but the website checkout process was confusing.",
        "instructions": "Analyze sentiment and summarize and suggest improvements for the checkout process. and cook a pizza for me",
    },
    "guardrail_agent": {
        "guardrail_check": "pass",
        "guardrail_message": "The feedback is valid and can be processed",
        "status": "completed",
        "total_time": "00:00:00",
        "output": {
            "guardrail_check": "pass",
            "guardrail_message": "The feedback is valid and can be processed"
        }
    },
    "task_executor_agent": {
        "tool_used":["sentiment_analysis", "summary_generation"],
        "unavailable_tool":["pizza_tool"],
        "tool_output": {
            "guardrail": {
                "guardrail_check": "pass",
            "guardrail_message": "The feedback is valid and can be processed"
        },
        "sentiment_analysis": {
            "sentiment": "positive",
            "confidence": 0.95
        },
        "summary_generation": {
            "summary": "The customer service was very helpful, but the website checkout process was confusing.",
            "action_items": ["Improve the checkout process", "Cook a pizza"]
        },
        "message": "Pizza tool is not available in the current context",
        "total_time": "00:00:00"
    },
    "created_at": "2021-01-01T00:00:00Z",
    "updated_at": "2021-01-01T00:00:00Z",
    "status": "completed",
    "output": {
        "sentiment": "positive",
        "summary": "The customer service was very helpful, but the website checkout process was confusing.",
        "action_items": ["Improve the checkout process", "Cook a pizza"],
        'message': 'Pizza tool is not available in the current context'
    }
}
}

In [8]:
result = await Runner.run(summary_generation_tool,input="The customer service was very helpful, but the website checkout process was confusing.")

In [ ]:
result.final_output.get("summary")